In [4]:
import numpy as np

def sinkhorn_knopp(matrix, tol=1e-10, max_iter=1000):
    """
    Normalize a matrix using the Sinkhorn-Knopp algorithm.

    Parameters:
    - matrix: The input matrix to normalize.
    - tol: Tolerance for the stopping criterion.
    - max_iter: Maximum number of iterations.

    Returns:
    - The normalized matrix.
    """
    R, C = matrix.shape
    for _ in range(max_iter):
        # Row normalization
        row_sums = matrix.sum(axis=1, keepdims=True)
        matrix = matrix / row_sums
        
        # Column normalization
        col_sums = matrix.sum(axis=0, keepdims=True)
        matrix = matrix / col_sums
        
        # Check for convergence
        if np.max(np.abs(row_sums - 1)) < tol and np.max(np.abs(col_sums - 1)) < tol:
            break
            
    return matrix

# Example matrix
A = np.array([[1, 2], [4, 5], [7, 8]], dtype=float)

# Apply Sinkhorn-Knopp normalization
normalized_matrix = sinkhorn_knopp(A)

normalized_matrix


array([[0.27656019, 0.39010648],
       [0.35430765, 0.31235902],
       [0.36913216, 0.2975345 ]])

In [6]:

normalized_matrix.sum(axis=1, keepdims=True)

array([[0.66666667],
       [0.66666667],
       [0.66666667]])